# DS

## Clusterização

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/data.csv")

## Machine Learning

In [ ]:
pip install catboost --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

def run_ml_models(data, features, target):
    results = []

    # Convert categorical features into numeric features
    le = LabelEncoder()
    for feature in features:
        if data[feature].dtype == "object":
            data[feature] = le.fit_transform(data[feature])

    # Split the data into features and target variables
    X = data.drop(target, axis=1)
    y = data[target]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Models and names
    models = [
        ('DecisionTree', DecisionTreeClassifier(random_state=42)),
        ('RandomForest', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),
        ('SVM', Pipeline(steps=[('preprocessor', ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)])), ('classifier', SVC(kernel='linear', random_state=42))])),
        ('NaiveBayes', MultinomialNB()),
        ('KNN', KNeighborsClassifier(n_neighbors=5)),
        ('CatBoost', CatBoostClassifier(logging_level='Silent')),
        ('CatBoost_oversampled', CatBoostClassifier(logging_level='Silent'))
    ]

    for name, model in models:
        start_time = time.time()
        # Oversample only for CatBoost model
        if name == 'CatBoost_oversampled':
            # Split the data into categorical and numerical features
            X_cat = X[features].copy()
            X_num = X.drop(features, axis=1).copy()

            # Perform oversampling using SMOTE
            smote = SMOTE()
            X_train_cat, y_train_cat = smote.fit_resample(X_cat, y)

            # Concatenate categorical and numerical features
            X_train = pd.concat([X_train_cat.reset_index(drop=True), X_num.reset_index(drop=True)], axis=1)
            y_train = y_train_cat
        else:
            X_train = X_train
            y_train = y_train

        if name == 'NaiveBayes':
            preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)])
            X_train_transformed = csr_matrix(preprocessor.fit_transform(X_train))
            X_test_transformed = csr_matrix(preprocessor.transform(X_test))
            model.fit(X_train_transformed, y_train)
            y_pred = model.predict(X_test_transformed)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        end_time = time.time()
        processing_time = end_time - start_time
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        results.append([name, processing_time, precision, recall, f1])

    results_df = pd.DataFrame(results, columns=['Model', 'Processing Time', 'Precision', 'Recall', 'F1'])
    return results_df


# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/data.csv")
data = data[["idade", "genero", "estado_civil", "faixa_renda", "bloco"]]

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Run the ML models
results_df = run_ml_models(data, features, target)
print(results_df.sort_values(by='F1', ascending=False))


                  Model  Processing Time  Precision    Recall        F1
6  CatBoost_oversampled         7.941518   0.476005  0.347003  0.359413
5              CatBoost         7.902413   0.214038  0.277603  0.232204
0          DecisionTree         0.024220   0.212949  0.264984  0.225012
1          RandomForest         0.814208   0.187528  0.337539  0.223068
3            NaiveBayes         0.038788   0.191453  0.334385  0.221746
2                   SVM         0.260246   0.174412  0.321767  0.211367
4                   KNN         0.041692   0.193612  0.280757  0.211146


Justificativa para uso do parâmetro 'wighted':

- O parâmetro "weighted" na métrica F1-score calcula a média ponderada da precisão e do recall. Ele é adequado para casos de classificação multiclasse desbalanceados, onde cada classe pode ter um número diferente de amostras. Nesse caso, o F1-score ponderado dá maior peso às classes com menos amostras, enquanto ainda considera a precisão e o recall de todas as classes. Isso permite avaliar o desempenho geral do modelo de maneira mais equilibrada, levando em conta o desequilíbrio entre as classes.

Escolha do modelo:

- Com base nos resultados obtidos, é possível observar que o modelo que obteve o melhor desempenho foi o CatBoost_oversampled, que utilizou o algoritmo CatBoost e também fez uso de oversampling. Esse modelo apresentou uma precisão de 0.483, um recall de 0.365 e um F1-Score de 0.366. Isso significa que, levando em consideração todas as métricas, o modelo CatBoost_oversampled foi capaz de apresentar os melhores resultados na classificação dos dados, o que pode ser justificado pelo fato de ter sido utilizado o algoritmo CatBoost, que é conhecido por sua eficiência na modelagem de dados e classificação, além do uso de oversampling para lidar com o desbalanceamento das classes. Dessa forma, o uso do modelo CatBoost_oversampled pode ser considerado uma boa escolha para a tarefa de classificação dos dados.